In [ ]:
#!pip install pymongo

In [14]:
import os

CONNECTION_STRING = "mongodb+srv://<user>:<password>@genai-poc.mongocluster.cosmos.azure.com/?tls=true&authMechanism=SCRAM-SHA-256&retrywrites=false&maxIdleTimeMS=120000"
INDEX_NAME = "genaipoc-test-index"
NAMESPACE = "genaipoc_test_db.genaipoc_test_collection"
DB_NAME, COLLECTION_NAME = NAMESPACE.split(".")

In [15]:
#read .env file
from dotenv import load_dotenv
load_dotenv()

# os.getenv('OPENAI_API_KEY')
# os.getenv('OPENAI_API_VERSION')
# os.getenv('AZURE_OPENAI_ENDPOINT')
# os.environ[
#     "OPENAI_API_BASE"
# ] = "YOUR_OPEN_AI_ENDPOINT"  # https://example.openai.azure.com/
#os.getenv('OPENAI_API_KEY')
# os.environ[
#     "OPENAI_EMBEDDINGS_DEPLOYMENT"
# ] = "smart-agent-embedding-ada"  # the deployment name for the embedding model
#os.getenv("OPENAI_EMBEDDINGS_MODEL_NAME")



True

In [11]:
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.azure_cosmos_db import (
    AzureCosmosDBVectorSearch,
    CosmosDBSimilarityType,
)

In [12]:
SOURCE_FILE_NAME = "../../data/financebench/COCACOLA_2022_10K.pdf"

loader = PyPDFLoader(SOURCE_FILE_NAME)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=100)
docs = text_splitter.split_documents(documents)


In [17]:
# OpenAI Settings
model_deployment = os.getenv(
    "OPENAI_EMBEDDINGS_DEPLOYMENT", "smart-agent-embedding-ada"
)
model_name = os.getenv("OPENAI_EMBEDDINGS_MODEL_NAME", "text-embedding-ada-002")

# Load OpenAI access
import sys
sys.path.append(os.path.abspath('../../src'))
from azure_openai_conn import OpenAIembeddings

# openai_embeddings: OpenAIEmbeddings = OpenAIEmbeddings(
#     deployment=model_deployment, model=model_name, chunk_size=1
# )
embeddings = OpenAIembeddings()


In [18]:
from pymongo import MongoClient

INDEX_NAME = "genaipoc-test-index-2"
NAMESPACE = "genaipoc_test_db.genaipoc_test_collection"
DB_NAME, COLLECTION_NAME = NAMESPACE.split(".")

client: MongoClient = MongoClient(CONNECTION_STRING)
collection = client[DB_NAME][COLLECTION_NAME]


In [19]:
vectorstore = AzureCosmosDBVectorSearch.from_documents(
    docs,
    embeddings,
    collection=collection,
    index_name=INDEX_NAME,
)

num_lists = 100
dimensions = 1536
similarity_algorithm = CosmosDBSimilarityType.COS


ServerSelectionTimeoutError: azure cosmos db mongo vcore connection string:27017: [Errno -2] Name or service not known (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 658dbcc4fb1117761fd0c896, topology_type: Unknown, servers: [<ServerDescription ('azure cosmos db mongo vcore connection string', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('azure cosmos db mongo vcore connection string:27017: [Errno -2] Name or service not known (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

In [ ]:

vectorstore.create_index(num_lists, dimensions, similarity_algorithm)